### <h1 align="center">**Master Big Data et Intelligence Artificielle**</h1>
#### <h1 align="center">**Examen Pratique de Traitement d'Images**</h1>
##### <h1 align="center">**Session normale - 2023/2024**</h1>

### Réalisé par : LAGHJAJ ABDELLATIF
### Encadré par : Youssef ES-SAADY

In [169]:
import matplotlib.pyplot as plt 
import numpy as np
import os
import csv
import cv2
from PIL import Image
import dlib
import platform
import imutils
from imutils import face_utils
from scipy.spatial import distance as dist

La fonction plot_images permet d'afficher plusieurs images dans une grille en utilisant la bibliothèque Matplotlib. Elle prend en entrée une liste d'images, une liste optionnelle de titres pour chaque image, et une taille de figure facultative. La fonction calcule le nombre de lignes et de colonnes nécessaires pour afficher les images de manière compacte dans une grille. Ensuite, elle crée une figure avec des sous-graphiques Matplotlib et itère à travers ces sous-graphiques pour afficher chaque image avec son titre correspondant (si fourni). Enfin, elle ajuste l'espacement entre les sous-graphiques pour une présentation visuelle optimale et affiche la grille d'images.

In [170]:
def plot_images(images, titles=None, figsize=(15, 10)):
    """Display multiple images in a grid."""
    num_frames = len(images)
    rows = int(num_frames ** 0.5) + 1  # Calculate number of rows
    cols = (num_frames // rows) + 1    # Calculate number of columns
    
    if titles is None:
        titles = [''] * len(images)
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    for i, ax in enumerate(axes.flatten()):
        if i < len(images):
            ax.imshow(images[i])
            ax.set_title(titles[i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

### 1. Extraction des images à partir d'une vidéo.

La fonction extract_frames_at_intervals permet d'extraire des images à partir d'une vidéo à des intervalles réguliers. Elle prend en paramètres le chemin du fichier vidéo (video_path) et un intervalle de temps spécifié en secondes (interval). La fonction ouvre la vidéo, calcule la durée totale de la vidéo et extrait les images à des instants définis par l'intervalle.

La vidéo est lue frame par frame, en utilisant la fonction cv2.VideoCapture de la bibliothèque OpenCV. La position de la frame dans la vidéo est ajustée en fonction du temps spécifié par l'intervalle, puis la frame est ajoutée à une liste. Cette liste d'images extraites est ensuite renvoyée par la fonction.

La fonction gère également les erreurs potentielles liées à l'ouverture du fichier vidéo et à la lecture des frames, en affichant des messages d'erreur appropriés.

In [171]:
# Extract frames from a video pour chaque instant t
def extract_frames_at_intervals(video_path, interval = 10):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / cap.get(cv2.CAP_PROP_FPS)  # Duration in seconds

    frames = []
    for t in range(0, int(duration), interval):
        frame_time = t * total_frames / duration
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_time))
        
        ret, frame = cap.read()

        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            print(f"Error: Unable to read frame at {t} seconds.")

    cap.release()
    return frames

### 2. Conversion des images en niveaux de gris.

La fonction convert_to_grayscale prend une liste d'images au format BGR en entrée et retourne une liste d'images converties en niveaux de gris. Pour chaque image de la liste, la fonction utilise la bibliothèque OpenCV pour d'abord convertir l'image de BGR à RGB, puis la convertit de RGB à niveaux de gris. Le résultat est une liste d'images en niveaux de gris correspondantes à celles fournies en entrée.

In [172]:
def convert_to_grayscale(frames):
    # Convert each BGR image to grayscale
    images_gray = [cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.COLOR_RGB2GRAY) for image in frames]
    return images_gray

### 3. Suppression de bruits.

La fonction remove_noise permet de supprimer le bruit d'une ou plusieurs images en utilisant des filtres spécifiés. Elle prend en entrée une liste d'images ou une image unique, un paramètre de méthode ('gaussian' ou 'median') pour choisir le type de filtre, la taille du noyau du filtre, et l'écart-type (pour le filtre gaussien). La fonction retourne une liste d'images sans bruit, où le bruit a été atténué en fonction des paramètres spécifiés.

In [173]:
def remove_noise(images, method='gaussian', kernel_size=3, sigma=1.0):
    if not isinstance(images, list):
        images = [images]

    if method == 'gaussian':
        return [cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma) for image in images]
    elif method == 'median':
        return [cv2.medianBlur(image, kernel_size) for image in images]

### 4. Amélioration de la qualité des images (contraste, luminosité).

La fonction enhance_images améliore la qualité des images en appliquant le contraste adaptatif à l'aide de l'algorithme CLAHE (Contrast Limited Adaptive Histogram Equalization). Elle prend en entrée une liste d'images filtrées, les convertit en niveaux de gris, puis applique le CLAHE individuellement à chaque image en niveaux de gris. Le résultat est une liste d'images améliorées en contraste, prêtes à être affichées correctement en niveaux de gris sur un écran.

In [174]:
def enhance_images(images_filtered):
    # Convertir les images en niveaux de gris
    images_gray = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images_filtered]

    # Appliquer CLAHE à chaque image en niveaux de gris
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    images_enhanced = [clahe.apply(image) for image in images_gray]

    # Convertir les images en niveaux de gris en images BGR pour l'affichage correct
    images_bgr = [cv2.cvtColor(image_gray, cv2.COLOR_GRAY2BGR) for image_gray in images_enhanced]

    return images_bgr

### 5. Détection des contours (utilisant des méthodes telles que Canny ou Sobel).

La fonction detect_edges utilise l'opérateur de détection de contours Canny pour détecter les contours dans chaque image améliorée en contraste. Elle prend en entrée une liste d'images améliorées, applique l'algorithme Canny à chaque image individuellement avec des paramètres spécifiés, puis convertit les images résultantes en niveaux de gris en images BGR pour assurer un affichage correct. Le résultat final est une liste d'images affichant les contours détectés, prêtes à être visualisées.

In [175]:
def detect_edges(images_enhanced):
    # Appliquer Canny à chaque image
    images_edges = [cv2.Canny(image_enhanced, 100, 200) for image_enhanced in images_enhanced]
    
    #convertir les images en niveaux de gris en images BGR pour l'affichage correct
    images_edges = [cv2.cvtColor(image_edges, cv2.COLOR_GRAY2BGR) for image_edges in images_edges]
    return images_edges

### Sauvegarde des images traitées dans un dossier.

La fonction save_frames_to_folder permet d'enregistrer chaque image d'une liste de frames dans un dossier spécifié. Elle prend en entrée la liste des frames, le chemin du dossier de destination, et un préfixe optionnel pour le nom de chaque image. Si le dossier n'existe pas, la fonction le crée. Pour chaque frame, la fonction génère un nom d'image unique en fonction du préfixe et de l'indice de la frame, puis enregistre l'image convertie en format BGR dans le dossier. Ainsi, cette fonction facilite le stockage des frames extraites dans un emplacement spécifié sur le système de fichiers.

In [176]:
def save_frames_to_folder(frames, folder_path, image_prefix="frame"):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save each frame to the folder
    for i, frame in enumerate(frames):
        image_name = f"{image_prefix}_{i + 1}.png"
        image_path = os.path.join(folder_path, image_name)
        cv2.imwrite(image_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

### 6. Détection des yeux.

La fonction `eye_aspect_ratio` calcule le Rapport d'Aspect des Yeux (EAR) à partir des coordonnées des points caractéristiques du contour des yeux. Elle prend en entrée une liste de tuples représentant les coordonnées des points caractéristiques du contour des yeux, généralement obtenue à partir de la détection de visage et de la prédiction des points de repère. La formule pour calculer l'EAR est définie comme suit :

$$
\text{EAR} = \frac{(\text{p2} - \text{p6}) + (\text{p3} - \text{p5})}{2 \times (\text{p1} - \text{p4})}
$$

où p1, p2, p3, p4, p5 et p6 représentent des points caractéristiques spécifiques du contour des yeux, et $\text{dist.euclidean}$ est une fonction qui calcule la distance euclidienne entre deux points. Le résultat de cette fonction est l'EAR, souvent utilisé dans la détection de la somnolence en surveillant les changements de forme des yeux.

In [177]:
def eye_aspect_ratio(eye):
    p2_minus_p6 = dist.euclidean(eye[1], eye[5])
    p3_minus_p5 = dist.euclidean(eye[2], eye[4])
    p1_minus_p4 = dist.euclidean(eye[0], eye[3])
    ear = (p2_minus_p6 + p3_minus_p5) / (2.0 * p1_minus_p4)
    return ear

* **FACIAL_LANDMARK_PREDICTOR :** Chemin vers le prédicteur de points caractéristiques faciaux pré-entraîné de dlib.

* **MINIMUM_EAR :** Valeur minimale de l'EAR au-dessus de laquelle les yeux seront considérés comme ouverts, sinon fermés. Ce paramètre peut nécessiter un ajustement en fonction de vos besoins. Essayez de déterminer l'EAR dans différents scénarios, puis ajustez la valeur en conséquence. Notez également que cet EAR n'est pas pour un seul œil, mais l'EAR cumulé pour les deux yeux.

* **MAXIMUM_FRAME_COUNT :** La valeur de l'EAR change très rapidement. Même si vous clignez des yeux, l'EAR diminuera rapidement. Cependant, cligner des yeux ne signifie pas nécessairement la somnolence. La somnolence serait une situation où une personne a fermé les yeux (son EAR est très bas) pendant, disons, 10 images vidéo consécutives. Cette variable indique le nombre maximum d'images vidéo consécutives pendant lesquelles l'EAR peut rester inférieur à MINIMUM_EAR, sinon une alerte de somnolence est déclenchée.

In [178]:
FACIAL_LANDMARK_PREDICTOR = "shape_predictor_68_face_landmarks.dat"  
MINIMUM_EAR = 0.2
MAXIMUM_FRAME_COUNT = 10

Instancier le détecteur de visage faceDetector de dlib (qui détectera le visage dans une image) et le chercheur de points de repère landmarkFinder (qui trouvera 68 points de repère dans le visage détecté).

In [179]:
faceDetector = dlib.get_frontal_face_detector()
landmarkFinder = dlib.shape_predictor(FACIAL_LANDMARK_PREDICTOR)
landmarkPredictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
webcamFeed = cv2.VideoCapture(0)

nous trouvons les valeurs de début et de fin des identifiants de points de repère pour chaque œil. Vous pouvez le faire manuellement (37-42 pour l'œil droit et 43-48 pour l'œil gauche), mais en utilisant face_utils, vous pouvez obtenir ces valeurs en passant simplement le nom de l'œil.

In [180]:
(leftEyeStart, leftEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rightEyeStart, rightEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [181]:
def detect_eyes_ml(input_folder, output_folder):
    # Vérifier si le dossier de sortie existe, sinon le créer
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialiser le détecteur de visages de dlib
    face_detector = dlib.get_frontal_face_detector()
    
    # Initialiser le prédicteur de points de visage pour détecter les yeux
    eye_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    # Liste des fichiers dans le dossier d'entrée
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        # Chemin complet de l'image d'entrée
        input_path = os.path.join(input_folder, image_file)

        # Lire l'image
        image = cv2.imread(input_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Détecter les visages dans l'image
        faces = face_detector(gray_image)

        # Dessiner des rectangles autour des yeux détectés
        image_with_eyes = image.copy()
        for face in faces:
            landmarks = eye_predictor(gray_image, face)
            for n in range(36, 48):  # Les indices 36 à 47 représentent les yeux dans les landmarks
                x, y = landmarks.part(n).x, landmarks.part(n).y
                cv2.circle(image_with_eyes, (x, y), 2, (0, 255, 0), -1)

        # Chemin complet de l'image de sortie
        output_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_eyes_detected_ml.jpg")

        # Enregistrer l'image avec les yeux détectés
        cv2.imwrite(output_path, image_with_eyes)
        print(f"Yeux détectés avec modèle ML enregistrés : {output_path}")

### 7. Application de techniques de morphologie.

In [182]:
def erosion(frames, kernel_size=3, iterations=1):
    # Define the kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply erosion to each frame
    result_frames = [cv2.erode(frame, kernel, iterations=iterations) for frame in frames]
    
    return result_frames

def dilation(frames, kernel_size=3, iterations=1):
    # Define the kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply dilation to each frame
    result_frames = [cv2.dilate(frame, kernel, iterations=iterations) for frame in frames]
    
    return result_frames

def opening(frames, kernel_size=3, iterations=1):
    # Define the kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply opening (erosion followed by dilation) to each frame
    eroded_frames = [cv2.erode(frame, kernel, iterations=iterations) for frame in frames]
    result_frames = [cv2.dilate(eroded, kernel, iterations=iterations) for eroded in eroded_frames]
    
    return result_frames

def closing(frames, kernel_size=3, iterations=1):
    # Define the kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply closing (dilation followed by erosion) to each frame
    dilated_frames = [cv2.dilate(frame, kernel, iterations=iterations) for frame in frames]
    result_frames = [cv2.erode(dilated, kernel, iterations=iterations) for dilated in dilated_frames]
    
    return result_frames

### 8. Récupération des points d'intérêts (p1,...p6).

In [183]:
def eye_aspect_ratio(eye):
    # Convert the list of tuples to NumPy array
    eye = np.array(eye)
    
    # Vertical distances between eye landmarks
    v1 = np.linalg.norm(eye[1] - eye[5])
    v2 = np.linalg.norm(eye[2] - eye[4])
    
    # Horizontal distance between the outer eye corners
    h = np.linalg.norm(eye[0] - eye[3])
    
    # Eye aspect ratio
    ear = (v1 + v2) / (2 * h)
    return ear

def detect_eyes_and_save_images_and_ear(input_folder, output_folder):
    # Check if the output folder exists, create it if not
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize the dlib face detector
    face_detector = dlib.get_frontal_face_detector()

    # Initialize the dlib shape predictor for eye detection
    eye_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    # Create a CSV file to save EAR values
    csv_file_path = os.path.join(output_folder, "ear_values.csv")
    with open(csv_file_path, mode='w', newline='') as csvfile:
        fieldnames = ['Image', 'EAR']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Liste des fichiers dans le dossier d'entrée
        image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

        for image_file in image_files:
            # Chemin complet de l'image d'entrée
            input_path = os.path.join(input_folder, image_file)

            # Lire l'image
            image = cv2.imread(input_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Détecter les visages dans l'image
            faces = face_detector(gray_image)

            # Vérifier si des visages sont détectés
            if faces:
                # Créer une copie de l'image pour dessiner les cercles sans affecter l'originale
                image_with_eyes = image.copy()

                # Dessiner des cercles autour des yeux sur la copie de l'image
                for face in faces:
                    landmarks = eye_predictor(gray_image, face)
                    for n in range(36, 48):  # Les indices 36 à 47 représentent les yeux dans les landmarks
                        x, y = landmarks.part(n).x, landmarks.part(n).y
                        cv2.circle(image_with_eyes, (x, y), 2, (0, 255, 0), -1)

                # Chemin complet de l'image de sortie avec les yeux détectés
                output_path_with_eyes = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_with_eyes.jpg")
                cv2.imwrite(output_path_with_eyes, image_with_eyes)
                print(f"Image avec yeux détectés enregistrée : {output_path_with_eyes}")

                # Calculer et enregistrer l'EAR pour chaque œil dans l'image
                for face in faces:
                    landmarks = eye_predictor(gray_image, face)
                    left_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
                    right_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
                    
                    # Calculer l'EAR pour les yeux gauche et droit
                    ear_left = eye_aspect_ratio(left_eye)
                    ear_right = eye_aspect_ratio(right_eye)
                    
                    # EAR moyen des deux yeux
                    ear_avg = (ear_left + ear_right) / 2

                    # Enregistrer la valeur de l'EAR dans le fichier CSV
                    writer.writerow({'Image': image_file, 'EAR': ear_avg})

                    print(f"EAR calculé pour {image_file}: {ear_avg}")

### Test de l'application sur une vidéo.

In [184]:
# video_path = "videos/video-1.mp4"
# t = 10 # Extract frames every 10 seconds

# img_bruit = ajouter_bruit_impulsionnel(img, 0.1)
# frames = extract_frames_at_intervals(video_path, interval=t)
# save_frames_to_folder(frames, "input_frames", image_prefix="frame")
# gray_frames = convert_to_grayscale(frames)

# # Afficher les images extraites
# plot_images(frames, [f"Frame extract {i+1}" for i in range(len(frames))])

# # Afficher les images en niveaux de gris
# for i, image_gray in enumerate(gray_frames):
#     # Convert grayscale image to BGR for correct display
#     image_bgr = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2BGR)
#     gray_frames[i] = image_bgr
    
# # plot_images(gray_frames, [f"Gray Frame {i+1}" for i in range(len(gray_frames))])

# # Supprimer le bruit des images
# frames_without_noise = remove_noise(frames, method='gaussian', kernel_size=5, sigma=1.0)
# plot_images(frames_without_noise, [f"Frame without noise {i+1}" for i in range(len(frames_without_noise))])

# # Améliorer la qualité des images
# enhanced_frames = enhance_images(frames_without_noise)
# plot_images(enhanced_frames, [f"Enhanced Frame {i+1}" for i in range(len(enhanced_frames))])

# # Détecter les bords dans les images
# edges_frames = detect_edges(enhanced_frames)
# plot_images(edges_frames, [f"Edges Frame {i+1}" for i in range(len(edges_frames))])

# # Les opérations morphologiques
# output_folder = "morphologies_images"

# # Erosion
# eroded_frames = erosion(enhanced_frames, kernel_size=3, iterations=1)
# plot_images(eroded_frames, [f"Eroded Frame {i+1}" for i in range(len(eroded_frames))])
# save_frames_to_folder(frames=eroded_frames, folder_path=output_folder)

# # Dilation
# dilated_frames = dilation(enhanced_frames, kernel_size=3, iterations=1)
# plot_images(dilated_frames, [f"Dilated Frame {i+1}" for i in range(len(dilated_frames))])
# save_frames_to_folder(frames=dilated_frames, folder_path=output_folder)

# # Opening
# opened_frames = opening(enhanced_frames, kernel_size=3, iterations=1)
# save_frames_to_folder(frames=opened_frames, folder_path=output_folder)

# # Enregistrer les images dans un dossier
# output_folder = "output_frames"
# save_frames_to_folder(edges_frames, output_folder, image_prefix="frame_edges")

In [185]:
input_folder = "input_frames"
output_folder = "eyes_detected"

detect_eyes_ml(input_folder, output_folder)

Yeux détectés avec modèle ML enregistrés : eyes_detected/frame_1_eyes_detected_ml.jpg
Yeux détectés avec modèle ML enregistrés : eyes_detected/frame_3_eyes_detected_ml.jpg
Yeux détectés avec modèle ML enregistrés : eyes_detected/frame_2_eyes_detected_ml.jpg


In [186]:
input_folder = "morphologies_images"
output_csv = "ear_results"

detect_eyes_and_save_images_and_ear(input_folder, output_csv)

Image avec yeux détectés enregistrée : ear_results/frame_1_with_eyes.jpg
EAR calculé pour frame_1.png: 0.46194877289478753
EAR calculé pour frame_1.png: 0.4589423674778521
EAR calculé pour frame_1.png: 0.3491574212460923
Image avec yeux détectés enregistrée : ear_results/frame_3_with_eyes.jpg
EAR calculé pour frame_3.png: 0.4120040431297233
EAR calculé pour frame_3.png: 0.4250110042739885
EAR calculé pour frame_3.png: 0.4218599905051843
Image avec yeux détectés enregistrée : ear_results/frame_2_with_eyes.jpg
EAR calculé pour frame_2.png: 0.4586299078608632
EAR calculé pour frame_2.png: 0.38387322171731875
EAR calculé pour frame_2.png: 0.4332454907937815


In [187]:
# Initialize frame counter
EYE_CLOSED_COUNTER = 0

# Set the window name
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)

# Set the desired window size
cv2.resizeWindow("Frame", 600, 600)  # Adjust the size as needed

try:
    while True:
        # Read the frame from the webcam feed
        (status, image) = webcamFeed.read()
        image = imutils.resize(image, width=800)
        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = faceDetector(grayImage, 0)

        for face in faces:
            # Predict facial landmarks
            faceLandmarks = landmarkPredictor(grayImage, face)
            faceLandmarks = face_utils.shape_to_np(faceLandmarks)

            # Extract left and right eyes
            leftEye = faceLandmarks[leftEyeStart:leftEyeEnd]
            rightEye = faceLandmarks[rightEyeStart:rightEyeEnd]

            # Calculate eye aspect ratio (EAR)
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Draw contours around eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(image, [leftEyeHull], -1, (255, 0, 0), 2)
            cv2.drawContours(image, [rightEyeHull], -1, (255, 0, 0), 2)

            # Check for drowsiness
            if ear < MINIMUM_EAR:
                EYE_CLOSED_COUNTER += 1
            else:
                EYE_CLOSED_COUNTER = 0

            # Display EAR value on the frame
            cv2.putText(image, f"EAR: {round(ear, 2)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Check if drowsiness is detected
            if EYE_CLOSED_COUNTER >= MAXIMUM_FRAME_COUNT:
                cv2.putText(image, "Drowsiness", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Play alert sound (system beep)
                if platform.system() == "Windows":
                    os.system("rundll32.exe user32.dll,MessageBeep -1")
                else:
                    pass
        
        # Display the frame
        cv2.imshow("Frame", image)
        
        # Wait for a key press
        key = cv2.waitKey(1) & 0xFF
        
        # Break the loop if 'q' is pressed
        if key == ord("q"):
            break

except Exception as e:
    print(f"An error occurred: {e}")

# Release the webcam feed and close all windows
webcamFeed.release()
cv2.destroyAllWindows()

In [188]:
# Initialize frame counter
EYE_CLOSED_COUNTER = 0
video_path = "videos/video-2.mp4"
webcamFeed = cv2.VideoCapture(video_path)

# Specify the desired width for the displayed frame
desired_width = 360

try:
    while True:
        # Read the frame from the webcam feed
        (status, image) = webcamFeed.read()

        if not status:
            break  # Break the loop if the video has ended

        # Resize the frame to the desired width
        image = imutils.resize(image, width=desired_width)

        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = faceDetector(grayImage, 0)

        for face in faces:
            # Predict facial landmarks
            faceLandmarks = landmarkPredictor(grayImage, face)
            faceLandmarks = face_utils.shape_to_np(faceLandmarks)

            # Extract left and right eyes
            leftEye = faceLandmarks[leftEyeStart:leftEyeEnd]
            rightEye = faceLandmarks[rightEyeStart:rightEyeEnd]

            # Calculate eye aspect ratio (EAR)
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Draw contours around eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(image, [leftEyeHull], -1, (255, 0, 0), 2)
            cv2.drawContours(image, [rightEyeHull], -1, (255, 0, 0), 2)

            # Check for drowsiness
            if ear < MINIMUM_EAR:
                EYE_CLOSED_COUNTER += 1
            else:
                EYE_CLOSED_COUNTER = 0

            # Display EAR value on the frame
            cv2.putText(image, f"EAR: {round(ear, 2)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Check if drowsiness is detected
            if EYE_CLOSED_COUNTER >= MAXIMUM_FRAME_COUNT:
                cv2.putText(image, "Drowsiness", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                # Play alert sound (system beep)
                if platform.system() == "Windows":
                    os.system("rundll32.exe user32.dll,MessageBeep -1")
                else:
                    pass
        
        # Display the frame
        cv2.imshow("Frame", image)

        # Wait for a key press
        key = cv2.waitKey(1) & 0xFF
        
        # Break the loop if 'q' is pressed
        if key == ord("q"):
            break

except Exception as e:
    print(f"An error occurred: {e}")

# Release the webcam feed and close all windows
webcamFeed.release()
cv2.destroyAllWindows()

SyntaxError: invalid syntax (657700636.py, line 62)